# PedSen Initial ArchR Setup

In [ ]:
quiet_library <- function(...) { suppressPackageStartupMessages(library(...)) }
quiet_library(ArchR)
quiet_library(hise)
quiet_library(Seurat)
quiet_library(ggplot2)
quiet_library(Matrix)
quiet_library(H5weaver)
quiet_library(dplyr)
#quiet_library(viridis)
#quiet_library(harmony)
#quiet_library(Nebulosa)
quiet_library(stringr)

In [ ]:
addArchRThreads(32)
addArchRGenome("hg38")

# Setup

In [ ]:
ped_arrows <- c('/home/jupyter/Organized_Gating_Analysis/ped_vs_senior_arrow/B076-P1_PB00124-02.arrow',
                '/home/jupyter/Organized_Gating_Analysis/ped_vs_senior_arrow/B069-P1_PB00172-02.arrow',
                '/home/jupyter/Organized_Gating_Analysis/ped_vs_senior_arrow/B069-P1_PB00182-02.arrow',
                '/home/jupyter/Organized_Gating_Analysis/ped_vs_senior_arrow/B076-P1_PB00127-02.arrow',
                '/home/jupyter/Organized_Gating_Analysis/ped_vs_senior_arrow/B069-P1_PB00807-02.arrow',
                '/home/jupyter/Organized_Gating_Analysis/ped_vs_senior_arrow/B065-P1_PB00173-02.arrow',
                '/home/jupyter/Organized_Gating_Analysis/ped_vs_senior_arrow/B065-P1_PB00192-02.arrow',
                '/home/jupyter/Organized_Gating_Analysis/ped_vs_senior_arrow/B065-P1_PB00197-02.arrow')

In [ ]:
sen_arrows <- c('/home/jupyter/Organized_Gating_Analysis/ped_vs_senior_arrow/B065-P1_PB00593-04.arrow',
                '/home/jupyter/Organized_Gating_Analysis/ped_vs_senior_arrow/B069-P1_PB00323-02.arrow',
                '/home/jupyter/Organized_Gating_Analysis/ped_vs_senior_arrow/B076-P1_PB00368-04.arrow',
                '/home/jupyter/Organized_Gating_Analysis/ped_vs_senior_arrow/B076-P1_PB00353-03.arrow',
                '/home/jupyter/Organized_Gating_Analysis/ped_vs_senior_arrow/B076-P1_PB00334-03.arrow',
                '/home/jupyter/Organized_Gating_Analysis/ped_vs_senior_arrow/B076-P1_PB00377-03.arrow',
                '/home/jupyter/Organized_Gating_Analysis/ped_vs_senior_arrow/B076-P1_PB00545-02.arrow',
                '/home/jupyter/Organized_Gating_Analysis/ped_vs_senior_arrow/B076-P1_PB00599-02.arrow')

## Create ArchR Project

In [ ]:
proj <- ArchRProject(
  ArrowFiles = c(ped_arrows, sen_arrows), 
  outputDirectory = "/home/jupyter/Organized_Gating_Analysis/08_Rework/Figure5/",
  copyArrows = TRUE #This is recommened so that if you modify the Arrow files you have an original copy for later usage.
)

In [ ]:
proj

## Subset ArchR Project by barcodes from RNA+ADT object

In [ ]:
rna_adt_so <- readRDS(file = '/home/jupyter/Organized_Gating_Analysis/08_Rework/Figure5/mnp_merge_updated2.rds')

In [ ]:
head(proj$cellNames)
clean_barcodes <- str_sub(proj$cellNames, 20, 51)
head(clean_barcodes)

In [ ]:
proj$cellNames_clean <- clean_barcodes

In [ ]:
barcodes_rna <- colnames(rna_adt_so[['RNA']])
head(barcodes_rna)

In [ ]:
idx_new <- which(proj$cellNames_clean %in% barcodes_rna)
head(idx_new)

In [ ]:
length(colnames(rna_adt_so[['RNA']]))
length(proj$cellNames)

In [ ]:
cells_subset <- proj$cellNames[idx_new]
proj_subset <- proj[cells_subset,]
proj_subset

## Transfer Metadata to ArchR Project

In [ ]:
rna_metadata <- rna_adt_so@meta.data
head(rna_metadata)
head(proj_subset$cellNames_clean)

In [ ]:
bc_idx <- barcodes_rna %in% clean_barcodes
table(bc_idx)

In [ ]:
# all cells overlap - so no need to subset rna metadata
# rna_meta_subset <- rna_metadata[bc_idx,]
# length(rownames(rna_meta_subset))

In [ ]:
# check barcode alignment
table(proj_subset$cellNames_clean == rownames(rna_metadata))

In [ ]:
head(rna_metadata)

In [ ]:
rna_metadata <- rna_metadata[match(proj_subset$cellNames_clean, rownames(rna_metadata)),]

In [ ]:
head(rna_metadata)
head(proj_subset$cellNames_clean)

In [ ]:
# check re-aligned barcodes - should be all true
table(proj_subset$cellNames_clean == rownames(rna_metadata))

### Add metadata

In [ ]:
colnames(rna_metadata)

In [ ]:
proj_subset$cell.types <- rna_metadata$mnp_analysis
proj_subset$gating_celltype <- rna_metadata$gating_celltype
proj_subset$pediatric_senior <- rna_metadata$pediatric_senior
proj_subset$CMV <- rna_metadata$CMV
proj_subset$age_cmv <- rna_metadata$age_cmv
proj_subset$predicted.t_celltype.l1 <- rna_metadata$predicted.t_celltype.l1
proj_subset$predicted.t_celltype.l2 <- rna_metadata$predicted.t_celltype.l2
proj_subset$predicted.t_celltype.l3 <- rna_metadata$predicted.t_celltype.l3
proj_subset$batch_id <- rna_metadata$batch_id
proj_subset$pbmc_sample_id <- rna_metadata$pbmc_sample_id

## Add Gene Expression to ArchR Project

In [ ]:
library(SingleCellExperiment)

In [ ]:
rna_mtx <- as.sparse(rna_adt_so@assays$RNA@counts)

In [ ]:
head(colnames(rna_mtx))
head(proj$cellNames)

In [ ]:
archr_style_barcodes <- paste(rna_adt_so$batch_id, "-P1_",rna_adt_so$pbmc_sample_id,"#",colnames(rna_mtx), sep = "")
head(archr_style_barcodes)

In [ ]:
table(archr_style_barcodes %in% proj$cellNames)
rna_adt_so$ATAC_Cell_ID <- archr_style_barcodes

In [ ]:
colnames(rna_mtx) <- archr_style_barcodes

In [ ]:
atac_genes <- getGenes(ArchRProj = proj)
head(atac_genes)

In [ ]:
genes <- getGenes(proj, symbols = rownames(rna_mtx))

In [ ]:
genes

In [ ]:
idx <- rownames(rna_mtx) %in% atac_genes$symbol
head(idx)

In [ ]:
rna_mtx_subset <- rna_mtx[idx,]

In [ ]:
sce <- SingleCellExperiment(list(counts = rna_mtx_subset),rowRanges= genes)
sce

In [ ]:
proj_subset <- addGeneExpressionMatrix(input = proj_subset, seRNA = sce)

# Save ArchR Project

In [ ]:
saveArchRProject(ArchRProj = proj_subset, outputDirectory = '/home/jupyter/Organized_Gating_Analysis/08_Rework/Figure5/', load = F)

In [ ]:
getAvailableMatrices(proj_subset)

# Cluster

In [ ]:
proj_subset <- addIterativeLSI(proj_subset, name = 'IterativeLSI', force = TRUE, varFeatures = 75000)

In [ ]:
proj_subset <- addHarmony(proj_subset, reducedDims = 'IterativeLSI', groupBy = 'batch_id', force = TRUE)

In [ ]:
# proj_subset <- addClusters(proj_subset, reducedDims = 'IterativeLSI', name = 'Clusters', force = TRUE, resolution = 0.2)
proj_subset <- addClusters(proj_subset, reducedDims = 'Harmony', name = 'Clusters', force = TRUE, resolution = 0.5)

In [ ]:
# proj_subset <- addUMAP(proj_subset, reducedDims = 'IterativeLSI', name = 'UMAP', force = TRUE)
proj_subset <- addUMAP(proj_subset, reducedDims = 'Harmony', name = 'UMAP', force = TRUE)

In [ ]:
options(repr.plosp_mergeidth = 8, repr.plot.height = 8)
plotEmbedding(ArchRProj = proj_subset, colorBy = "cellColData", name = "Clusters", embedding = "UMAP")
plotEmbedding(ArchRProj = proj_subset, colorBy = "cellColData", name = "pediatric_senior", embedding = "UMAP")
plotEmbedding(ArchRProj = proj_subset, colorBy = "cellColData", name = "cell.types", embedding = "UMAP")

In [ ]:
markersGS <- getMarkerFeatures(
    ArchRProj = proj_subset, 
    useMatrix = "GeneScoreMatrix", 
    groupBy = "Clusters",
    bias = c("TSSEnrichment", "log10(nFrags)"),
    testMethod = "wilcoxon"
)

# Peak Calling

In [ ]:
proj_subset <- addGroupCoverages(ArchRProj = proj_subset, groupBy = "Clusters")

In [ ]:
pathToMacs2 <- findMacs2()

In [ ]:
proj_subset <- addReproduciblePeakSet(
    ArchRProj = proj_subset, 
    groupBy = "Clusters", 
    pathToMacs2 = pathToMacs2
)

In [ ]:
proj_subset <- addPeakMatrix(proj_subset)

# Marker Peaks

In [ ]:
markersPeaks <- getMarkerFeatures(
    ArchRProj = proj_subset, 
    useMatrix = "PeakMatrix", 
    groupBy = "Clusters",
  bias = c("TSSEnrichment", "log10(nFrags)"),
  testMethod = "wilcoxon"
)

In [ ]:
heatmapPeaks <- markerHeatmap(
  seMarker = markersPeaks, 
  cutOff = "FDR <= 0.1 & Log2FC >= 0.5",
  transpose = TRUE
)

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 8)
draw(heatmapPeaks, heatmap_legend_side = "bot", annotation_legend_side = "bot")

# Motifs

In [ ]:
proj_subset <- addMotifAnnotations(ArchRProj = proj_subset, motifSet = "JASPAR2020", name = "Motif")

In [ ]:
enrichMotifs <- peakAnnoEnrichment(
    seMarker = markersPeaks,
    ArchRProj = proj_subset,
    peakAnnotation = "Motif",
    cutOff = "FDR <= 0.1 & Log2FC >= 0.5"
  )

In [ ]:
heatmapEM <- plotEnrichHeatmap(enrichMotifs, n = 7, transpose = TRUE)

In [ ]:
ComplexHeatmap::draw(heatmapEM, heatmap_legend_side = "bot", annotation_legend_side = "bot")

# ChromVar

In [ ]:
proj_subset <- addBgdPeaks(proj_subset)

In [ ]:
proj_subset <- addDeviationsMatrix(
  ArchRProj = proj_subset, 
  peakAnnotation = "Motif",
  force = TRUE
)

In [ ]:
motifs <- c("LEF1", "SOX4", "TCF7", "EOMES", "TBX21", "RORC","ZNF683")
markerMotifs <- getFeatures(proj_subset, select = paste(motifs, collapse="|"), useMatrix = "MotifMatrix")
markerMotifs

In [ ]:
markerMotifs <- grep("z:", markerMotifs, value = TRUE)
# markerMotifs <- markerMotifs[markerMotifs %ni% "z:SREBF1_22"]
markerMotifs

In [ ]:
proj_subset <- addImputeWeights(proj_subset)

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 8)
p <- plotEmbedding(
    ArchRProj = proj_subset, 
    colorBy = "MotifMatrix", 
    name = sort(markerMotifs), 
    embedding = "UMAP",
    imputeWeights = getImputeWeights(proj_subset)
)

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 18)
p2 <- lapply(p, function(x){
    x + guides(color = FALSE, fill = FALSE) + 
    theme_ArchR(baseSize = 6.5) +
    theme(plot.margin = unit(c(0, 0, 0, 0), "cm")) +
    theme(
        axis.text.x=element_blank(), 
        axis.ticks.x=element_blank(), 
        axis.text.y=element_blank(), 
        axis.ticks.y=element_blank()
    )
})
do.call(cowplot::plot_grid, c(list(ncol = 3),p2))

# Save

In [ ]:
saveArchRProject(ArchRProj = proj_subset, outputDirectory = '/home/jupyter/Organized_Gating_Analysis/08_Rework/Figure5/', load = F)

In [ ]:
sessionInfo()